In [1]:
import numpy as np
import pandas as pd
import datetime
import math

In [16]:
dam = pd.read_csv('dam_out.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

year  month  day  market   price  volume  day_of_week  \
date       hour                                                          
2016-01-01 0     2016      1    1       1   40.10   477.5            4   
           1     2016      1    1       1   40.08   486.6            4   
           2     2016      1    1       1   40.06   495.6            4   
           3     2016      1    1       1   40.08   496.6            4   
           4     2016      1    1       1   40.07   502.9            4   
...               ...    ...  ...     ...     ...     ...          ...   
2021-04-29 4     2021      4   29       1   39.97   126.1            3   
           12    2021      4   29       1   68.06    71.0            3   
2021-04-30 0     2021      4   30       1   44.04    59.6            4   
           8     2021      4   30       1  190.00    72.0            4   
           17    2021      4   30       1   67.97   114.2            4   

                     value  price_min  price_max  price_mean  price_standard  \
date       hour                                                                
2016-01-01 0     19147.750      35.09      60.08    45.98250         45.8275   
           1     19502.928      35.09      60.08    45.98250         45.8275   
           2     19853.736      35.09      60.08    45.98250         45.8275   
           3     19903.728      35.09      60.08    45.98250         45.8275   
           4     20151.203      35.09      60.08    45.98250         45.8275   
...                    ...        ...        ...         ...             ...   
2021-04-29 4      5040.217      39.79     296.82    98.74250         68.0600   
           12     4832.260      39.79     296.82    98.74250         68.0600   
2021-04-30 0      2624.784      38.97     295.98    98.21875         67.9825   
           8     13680.000      38.97     295.98    98.21875         67.9825   
           17     7762.174      38.97     295.98    98.21875         67.9825   

                 price_morning_max  price_evening_max       cat  
date       hour                                                  
2016-01-01 0                 48.03              60.08       off  
           1                 48.03              60.08       off  
           2                 48.03              60.08       off  
           3                 48.03              60.08       off  
           4                 48.03              60.08       off  
...                            ...                ...       ...  
2021-04-29 4                195.01             296.82       off  
           12               195.01             296.82  standard  
2021-04-30 0                190.00             295.98       off  
           8                190.00             295.98   morning  
           17               190.00             295.98  standard  

[46728 rows x 15 columns]

In [17]:
flow = pd.read_csv('flow.csv')

flow['date']=pd.to_datetime(flow['flowdate'],format="%Y-%m-%d")#"%d/%m/%Y")
flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.reset_index()
flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2016-01-02,13,50,BOT,ZIMA,2016,1,2,5,50.05,2502.50
1,2016-01-02,14,63,ZIMA,ZIM,2016,1,2,5,50.31,3169.53
2,2016-01-02,15,263,ZIM,ZAMZ,2016,1,2,5,50.49,13278.87
3,2016-01-02,16,100,MOZN_EDM,ZIMA,2016,1,2,5,56.07,5607.00
4,2016-01-02,17,120,ZIM,ZAMZ,2016,1,2,5,56.06,6727.20
...,...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,3,RSAN,RSAS,2021,3,31,2,148.01,444.03
333321,2021-03-31,20,13,RSAN,BOT,2021,3,31,2,139.06,1807.78
333322,2021-03-31,21,33,MOZS,RSAN,2021,3,31,2,89.03,2937.99
333323,2021-03-31,22,43,ZIMA,ZIM,2021,3,31,2,36.98,1590.14


In [28]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2016-01-02,13,50,2502.50,50.05,ZIMA,2016,1,2,5
1,2016-01-02,14,63,3169.53,50.31,ZIM,2016,1,2,5
2,2016-01-02,15,263,13278.87,50.49,ZAMZ,2016,1,2,5
3,2016-01-02,16,100,5607.00,56.07,ZIMA,2016,1,2,5
4,2016-01-02,17,120,6727.20,56.06,ZAMZ,2016,1,2,5
...,...,...,...,...,...,...,...,...,...,...
333320,2021-03-31,19,-3,-444.03,148.01,RSAN,2021,3,31,2
333321,2021-03-31,20,-13,-1807.78,139.06,RSAN,2021,3,31,2
333322,2021-03-31,21,-33,-2937.99,89.03,MOZS,2021,3,31,2
333323,2021-03-31,22,-43,-1590.14,36.98,ZIMA,2021,3,31,2


In [29]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='MOZ'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [30]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows

,date,hour,node,year,month,day,day_of_week,index,flow,value,price
0,2016-01-01,0,ZAMC,2016,1,1,4,5607,160,6416.00,40.10
1,2016-01-01,0,ZAMZ,2016,1,1,4,11320,10,401.00,80.20
2,2016-01-01,0,ZIM,2016,1,1,4,5713,-170,-6817.00,40.10
3,2016-01-01,1,ZAMC,2016,1,1,4,5714,160,6412.80,40.08
4,2016-01-01,1,ZAMZ,2016,1,1,4,11322,10,400.80,80.16
...,...,...,...,...,...,...,...,...,...,...,...
289677,2021-03-31,22,ZAMC,2021,3,31,2,333292,43,1590.14,36.98
289680,2021-03-31,23,BOT,2021,3,31,2,666156,81,2993.76,73.92
289681,2021-03-31,23,MOZ,2021,3,31,2,666292,-164,-6061.44,73.92
289682,2021-03-31,23,NAM,2021,3,31,2,333309,42,1552.32,36.96


In [31]:
flows=flows.set_index(['date','hour'])

In [32]:
flows['cat']=dam['cat']
flows.reset_index()

,date,hour,node,year,month,day,day_of_week,index,flow,value,price,cat
0,2016-01-01,0,ZAMC,2016,1,1,4,5607,160,6416.00,40.10,off
1,2016-01-01,0,ZAMZ,2016,1,1,4,11320,10,401.00,80.20,off
2,2016-01-01,0,ZIM,2016,1,1,4,5713,-170,-6817.00,40.10,off
3,2016-01-01,1,ZAMC,2016,1,1,4,5714,160,6412.80,40.08,off
4,2016-01-01,1,ZAMZ,2016,1,1,4,11322,10,400.80,80.16,off
...,...,...,...,...,...,...,...,...,...,...,...,...
188381,2021-03-31,22,ZAMC,2021,3,31,2,333292,43,1590.14,36.98,off
188382,2021-03-31,23,BOT,2021,3,31,2,666156,81,2993.76,73.92,off
188383,2021-03-31,23,MOZ,2021,3,31,2,666292,-164,-6061.44,73.92,off
188384,2021-03-31,23,NAM,2021,3,31,2,333309,42,1552.32,36.96,off


In [41]:
flows.groupby(['year','month','hour', 'node']).sum()

day  day_of_week     index  flow     value    price
year month hour node                                                     
2016 1     0    BOT    83           14     17300   -91  -4357.17   241.26
                MOZ   402           63    197289 -1829 -90774.59  3142.85
                NAM    25            3      4724    50   2409.80    96.36
                RSA    80           15     21724  -191  -9184.91   338.73
                ZAMC  382           82     71792  1965  92162.31  1254.85
...                   ...          ...       ...   ...       ...      ...
2021 3     23   NAM   494           86   9928333  1988  86941.09  1291.38
                RSA    37            9   9215638  -214 -10236.44  1340.44
                ZAMC  496           87  10253558  1955  85881.90  1339.63
                ZAMZ  137           12   2987557  -366 -15998.37   361.07
                ZIM    23            4   8226970    -3   -144.36  1203.00

[10113 rows x 6 columns]

In [43]:
#flow_monthly=flows.groupby(['year','month','hour', 'node']).sum()
#flow_monthly.rename(columns={'flow': 'flow_sum'}, inplace=True)
#flow_monthly=flow_monthly.reset_index().drop(['day_of_week','index'],axis=1)
#flow_monthly['price_weight_mean'] = flow_monthly['value'] / flow_monthly['flow_sum']
#flows.groupby(['year','month','hour', 'node']).mean()
#flow_monthly

In [44]:
#flow_monthly.to_csv('dam_flow_monthly.csv')

In [50]:
flow_monthly_cat=flows.groupby(['year','month', 'node','cat']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)

flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flows.groupby(['year','month', 'node','cat']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat

flow_sum       value  price_weight_mean  price_mean
year month node cat                                                          
2016 1     BOT  off          -2335  -108586.09          46.503679   47.300172
                standard      -410   -24771.25          60.417683   72.776364
           MOZ  evening       -153   -17232.93         112.633529  341.910000
                off         -23197 -1186113.71          51.132203  161.987312
                standard     -4575  -429298.28          93.835690  291.652895
...                            ...         ...                ...         ...
2021 3     ZAMZ standard    -12689  -855548.35          67.424411  126.491495
           ZIM  evening          0        0.00                NaN  850.050000
                morning       -214   -19775.24          92.407664  387.655000
                off            -49    -2788.05          56.898980  148.219000
                standard      -331   -31618.04          95.522779  436.733636

[1607 rows x 4 columns]

In [248]:
flow_monthly_cat.to_csv('dam_flow_monthly_cat.csv')